<a href="https://colab.research.google.com/github/swartn/cmip6-test/blob/master/cmip6_fgco2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Environment setup
! pip install --upgrade xarray zarr gcsfs cftime nc-time-axis bokeh intake-esm cmip6_preprocessing 

     |████████████████████████████████| 655kB 2.8MB/s 
     |████████████████████████████████| 3.3MB 53.9MB/s 
     |████████████████████████████████| 317kB 50.4MB/s 
Requirement already up-to-date: bokeh in /usr/local/lib/python3.6/dist-packages (1.4.0)
     |████████████████████████████████| 3.8MB 45.2MB/s 
     |████████████████████████████████| 40kB 4.4MB/s 
     |████████████████████████████████| 3.1MB 32.1MB/s 
     |████████████████████████████████| 4.1MB 26.4MB/s 
  Created wheel for zarr: filename=zarr-2.4.0-cp36-none-any.whl size=127066 sha256=e48acfec19ffbe7df23d540d6c079668de0c1c6d56dd21b7de4235ede4ab5ce2
  Stored in directory: /root/.cache/pip/wheels/e1/5b/25/24c685604b91139aba00a5b6299b53e7a0661f737f27782559
  Created wheel for asciitree: filename=asciitree-0.3.3-cp36-none-any.whl size=5037 sha256=d873b67a21f928bd5d8f4883c980eb2807e80b381db822db43b800c4bb08aafa
  Stored in directory: /root/.cache/pip/wheels/1d/d9/58/9808b306744df0208fccc640d3d9952a5bc7468502d42897d5
  Cre

In [3]:
!wget https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

--2020-03-06 05:28:52--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   183MB/s    in 0.3s    

2020-03-06 05:28:52 (183 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-

In [0]:
!conda install -y -c conda-forge esmpy=7.1.0 xesmf

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

In [0]:
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import matplotlib.pyplot as plt
import gcsfs
import intake
import xesmf as xe
%matplotlib inline

In [0]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

In [0]:
from cmip6_preprocessing.preprocessing import combined_preprocessing, replace_x_y_nominal_lat_lon, rename_cmip6

def wrapper(ds):
    ds = ds.copy()
    ds = rename_cmip6(ds)
    ds = replace_x_y_nominal_lat_lon(ds)
    return ds

In [0]:

# lets load a bunch more models this time
# load a few models to illustrate the problem
query = dict(experiment_id=['ssp585', 'historical'], table_id='Omon', 
             variable_id='fgco2', grid_label=['gn'], member_id=['r1i1p1f1', 'r1i1p2f1'])
cat = col.search(**query)

print(cat.df['source_id'].unique())
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, preprocess=combined_preprocessing)

In [0]:
ssp585_dict = {} # dictionary that will hold spliced DataArrays
for name, ds in dset_dict.items(): # Loop through dictionary
    model_name = name.split(".")[2]
    if ('ssp585' not in name): continue
    ssp585_dict[model_name] = ds

his_dict = {} # dictionary that will hold spliced DataArrays
for name, ds in dset_dict.items(): # Loop through dictionary
    model_name = name.split(".")[2]
    if ('historical' not in name) or (model_name not in ssp585_dict.keys()): continue
    his_dict[model_name] = ds

In [0]:
print(his_dict.keys())
print(ssp585_dict.keys())

In [0]:
# setup a common 1 degree global grid 
ds_out = xe.util.grid_global(1, 1)
ds_out 

In [0]:
kgs_to_molyr=1000./12.*3600*24*365

In [0]:
his_dict['ACCESS-ESM1-5']['fgco2']

In [0]:
 rm_his_dict = {}
for name, ds in his_dict.items():
    print(name)
    regridder = xe.Regridder(ds, ds_out, 'bilinear', ignore_degenerate=True)
    ds_in = ds.sel(time=slice('1980-01-01','2014-12-31')).mean(dim='time')*kgs_to_molyr
    dsrm = regridder(ds_in).compute()
    rm_his_dict[name] = dsrm

In [0]:
fig, axa = plt.subplots(3,3, figsize=(12,12))
for i, (name, ds) in enumerate(rm_his_dict.items()):
    print(name)
    ax=axa.flat[i]
    ds['fgco2'].plot(ax=ax, cmap='RdBu_r')
    ax.set_title(name)
    

In [0]:
axa.flat[0]

**OLD attempt**

In [0]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

experiment_id = "historical" 
variable_id = 'fgco2' 
table_id = 'Omon' 
member_id = 'r1i1p1f1' 

canesm5_query = df.query("activity_id=='CMIP' & "\
                         f"table_id == '{table_id}' & "\
                         f"variable_id == '{variable_id}' & "\
                         f"experiment_id == '{experiment_id}' & "\
                         f"member_id == '{member_id}' & "\
                         f"source_id == 'CanESM5'" 
                        )

# this only needs to be created once
gcs = gcsfs.GCSFileSystem(token='anon')

# stack the data into an xarray
canesm5 = []
for zstore in canesm5_query.zstore.values:
  mapper = gcs.get_mapper(zstore)
  temp = xr.open_zarr(mapper, consolidated=True)
  member_id = zstore.split('/')[7]
  temp['member_id'] = member_id
  canesm5.append(temp)

df = xr.concat(canesm5, dim='member_id')

In [0]:
df

In [0]:
fgco2_lim= canesm5.sel(time=slice('1980-01-01', '1985-12-31')).mean(dim='time')['fgco2']

In [0]:
fgco2_lim.mean(dim='member_id').plot()

In [0]:
fgco2_lim.std(dim='member_id').plot(vmin=0, vmax=1e-9, cmap='Reds')

In [0]:
fgco2_std= canesm5.sel(time=slice('1980-01-01', '1985-12-31')).std(dim='time')['fgco2'].mean(dim='member_id').compute()

In [0]:
fgco2_std.plot(vmin=0, vmax=1e-9, cmap='Reds')

In [0]:
fgco2_std_ann = canesm5.sel(time=slice('1980-01-01', '1985-12-31')).resample(time='A').mean().std(dim='time')['fgco2'].mean(dim='member_id').compute()

In [0]:
fgco2_std_ann.plot(vmin=0, vmax=1e-9, cmap='Reds')